In [1]:
import numpy as np
import pandas as pd
import nltk

In [ ]:
business = pd.read_json("business.json", lines=True)
reviews = pd.read_json("review.json")
merged = pd.merge(business, reviews, how='inner', on=['business_id'])
newData = merged.groupby('business_id',as_index=False)[['text']].aggregate(lambda x: list(x))
final = pd.merge(business, newData, how='inner', on=['business_id'])
final.dropna(subset=['categories'], inplace=True)
restaurants = final[final.apply(lambda x: 'Restaurants' in x['categories'], axis=1)]
restaurants.to_csv("finalMergedData.csv")

In [2]:
data = pd.read_csv("finalMergedData.csv")

In [3]:
data.columns

Index(['Unnamed: 0', 'address', 'attributes', 'business_id', 'categories',
       'city', 'hours', 'is_open', 'latitude', 'longitude', 'name',
       'postal_code', 'review_count', 'stars', 'state', 'text'],
      dtype='object')

In [4]:
data = data.drop('Unnamed: 0', axis=1)
data

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state,text
0,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON,"[""Went for Dim Sum when it was super packed. T..."
1,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': ""u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC,"[""Lunch Review:\n\nWe pulled up at 11:45 am to..."
2,2450 E Indian School Rd,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",1Dfx3zM-rW4n-31KeC8sJg,"Restaurants, Breakfast & Brunch, Mexican, Taco...",Phoenix,"{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",1,33.495194,-112.028588,Taco Bell,85016,18,3.0,AZ,['This location is consistently lightning fast...
3,5981 Andrews Rd,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...",fweCYi8FmbJXHCqLnwuk8w,"Italian, Restaurants, Pizza, Chicken Wings",Mentor-on-the-Lake,"{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ...",1,41.708520,-81.359556,Marco's Pizza,44060,16,4.0,OH,['This place is b.s. They sent me a $10 gift c...
4,"1775 E Tropicana Ave, Ste 29","{'OutdoorSeating': 'False', 'BusinessAcceptsCr...",PZ-LZzSlhSe9utkQYU8pFg,"Restaurants, Italian",Las Vegas,NaN,0,36.100016,-115.128528,Carluccio's Tivoli Gardens,89119,40,4.0,NV,['I can\'t enough about this place as where to...
5,"Center Core - Food Court, Fl 3, Pittsburgh Int...","{'RestaurantsTakeOut': 'True', 'BusinessParkin...",1RHY4K3BD22FK7Cfftn8Mg,"Sandwiches, Salad, Restaurants, Burgers, Comfo...",Pittsburgh,NaN,1,40.496177,-80.246011,Marathon Diner,15231,35,4.0,PA,"['My ""breakfast sandwich"" was a pathetic excus..."
6,6055 E Lake Mead Blvd,"{'BikeParking': 'True', 'BusinessParking': ""{'...",tstimHoMcYbkSC4eBA1wEg,"Mexican, Restaurants, Patisserie/Cake Shop, Fo...",Las Vegas,"{'Monday': '11:0-21:0', 'Tuesday': '10:0-21:0'...",1,36.195615,-115.040529,Maria's Mexican Restaurant & Bakery,89156,184,4.5,NV,"[""Legit Mexican food from a small homey shop.\..."
7,1170 Queen Street W,"{'WiFi': ""u'no'"", 'BikeParking': 'True', 'Rest...",NDuUMJfrWk52RA-H-OtrpA,"Juice Bars & Smoothies, Food, Restaurants, Fas...",Toronto,"{'Monday': '8:0-21:0', 'Tuesday': '8:0-21:0', ...",1,43.642889,-79.425429,Bolt Fresh Bar,M6J 1J5,57,3.0,ON,"[""Not too shabby! This place is pretty good fo..."
8,1051 Bloor Street W,"{'BusinessParking': ""{'garage': False, 'street...",SP_YXIEwkFPPl_9anCYmpQ,"Restaurants, Nightlife, Breakfast & Brunch, Ve...",Toronto,"{'Tuesday': '9:0-18:0', 'Wednesday': '9:0-18:0...",0,43.660494,-79.432099,The Steady Cafe & Bar,M6H 1M4,29,3.5,ON,['Let me start by saying that I love this plac...
9,6401 Morrison Blvd,"{'Alcohol': ""u'full_bar'"", 'BusinessAcceptsCre...",BvYU3jvGd0TJ7IyZdfiN2Q,"Sandwiches, Italian, American (Traditional), A...",Charlotte,"{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",0,35.156338,-80.831878,Manzetti's Tavern,28211,16,3.5,NC,"[""I had low country crab cakes for lunch becau..."


In [27]:
myCategories = data['categories']
myCategories

0        Specialty Food, Restaurants, Dim Sum, Imported...
1                        Sushi Bars, Restaurants, Japanese
2        Restaurants, Breakfast & Brunch, Mexican, Taco...
3               Italian, Restaurants, Pizza, Chicken Wings
4                                     Restaurants, Italian
5        Sandwiches, Salad, Restaurants, Burgers, Comfo...
6        Mexican, Restaurants, Patisserie/Cake Shop, Fo...
7        Juice Bars & Smoothies, Food, Restaurants, Fas...
8        Restaurants, Nightlife, Breakfast & Brunch, Ve...
9        Sandwiches, Italian, American (Traditional), A...
10       Restaurants, Breakfast & Brunch, Bars, Modern ...
11       Coffee & Tea, Hookah Bars, Nightlife, Persian/...
12                               Fish & Chips, Restaurants
13       Nightlife, Bars, Polish, Modern European, Rest...
14                                     Restaurants, Korean
15                       Japanese, Sushi Bars, Restaurants
16       Fast Food, Food, Restaurants, Ice Cream & Froz.

In [31]:
restaurantCategories = ["Indian", "Mexican", "Chinese", "European", "Korean", "Italian", "Asian Fusion", 
                        "American (Traditional)","American (New)", "Japanese", "Thai", "Seafood", 
                        "Mediterranean", "French", "Middle Eastern"]
restaurantCategories

['Indian',
 'Mexican',
 'Chinese',
 'European',
 'Korean',
 'Italian',
 'Asian Fusion',
 'American (Traditional)',
 'American (New)',
 'Japanese',
 'Thai',
 'Seafood',
 'Mediterranean',
 'French',
 'Middle Eastern']

In [32]:
data.dtypes

address          object
attributes       object
business_id      object
categories       object
city             object
hours            object
is_open           int64
latitude        float64
longitude       float64
name             object
postal_code      object
review_count      int64
stars           float64
state            object
text             object
newCategory      object
dtype: object

In [33]:
def assignLabels(data, restaurantCategories):
    for i in range(len(data)):
        oldCategories = [x.strip() for x in data.loc[i,'categories'].split(',')]
        newCategories = [category for category in oldCategories if category in restaurantCategories]
        newCategories = ','.join(newCategories)
        if newCategories == '':
            data.loc[i,'newCategory'] = "other"
        else:
            data.loc[i,'newCategory'] = newCategories        
    return data

# data['newCategory'] = data['newCategory'].astype(object)
newData = assignLabels(data, restaurantCategories)       
newData

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state,text,newCategory
0,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON,"[""Went for Dim Sum when it was super packed. T...","Chinese,Seafood"
1,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': ""u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC,"[""Lunch Review:\n\nWe pulled up at 11:45 am to...",Japanese
2,2450 E Indian School Rd,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",1Dfx3zM-rW4n-31KeC8sJg,"Restaurants, Breakfast & Brunch, Mexican, Taco...",Phoenix,"{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",1,33.495194,-112.028588,Taco Bell,85016,18,3.0,AZ,['This location is consistently lightning fast...,Mexican
3,5981 Andrews Rd,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...",fweCYi8FmbJXHCqLnwuk8w,"Italian, Restaurants, Pizza, Chicken Wings",Mentor-on-the-Lake,"{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ...",1,41.708520,-81.359556,Marco's Pizza,44060,16,4.0,OH,['This place is b.s. They sent me a $10 gift c...,Italian
4,"1775 E Tropicana Ave, Ste 29","{'OutdoorSeating': 'False', 'BusinessAcceptsCr...",PZ-LZzSlhSe9utkQYU8pFg,"Restaurants, Italian",Las Vegas,NaN,0,36.100016,-115.128528,Carluccio's Tivoli Gardens,89119,40,4.0,NV,['I can\'t enough about this place as where to...,Italian
5,"Center Core - Food Court, Fl 3, Pittsburgh Int...","{'RestaurantsTakeOut': 'True', 'BusinessParkin...",1RHY4K3BD22FK7Cfftn8Mg,"Sandwiches, Salad, Restaurants, Burgers, Comfo...",Pittsburgh,NaN,1,40.496177,-80.246011,Marathon Diner,15231,35,4.0,PA,"['My ""breakfast sandwich"" was a pathetic excus...",other
6,6055 E Lake Mead Blvd,"{'BikeParking': 'True', 'BusinessParking': ""{'...",tstimHoMcYbkSC4eBA1wEg,"Mexican, Restaurants, Patisserie/Cake Shop, Fo...",Las Vegas,"{'Monday': '11:0-21:0', 'Tuesday': '10:0-21:0'...",1,36.195615,-115.040529,Maria's Mexican Restaurant & Bakery,89156,184,4.5,NV,"[""Legit Mexican food from a small homey shop.\...",Mexican
7,1170 Queen Street W,"{'WiFi': ""u'no'"", 'BikeParking': 'True', 'Rest...",NDuUMJfrWk52RA-H-OtrpA,"Juice Bars & Smoothies, Food, Restaurants, Fas...",Toronto,"{'Monday': '8:0-21:0', 'Tuesday': '8:0-21:0', ...",1,43.642889,-79.425429,Bolt Fresh Bar,M6J 1J5,57,3.0,ON,"[""Not too shabby! This place is pretty good fo...",other
8,1051 Bloor Street W,"{'BusinessParking': ""{'garage': False, 'street...",SP_YXIEwkFPPl_9anCYmpQ,"Restaurants, Nightlife, Breakfast & Brunch, Ve...",Toronto,"{'Tuesday': '9:0-18:0', 'Wednesday': '9:0-18:0...",0,43.660494,-79.432099,The Steady Cafe & Bar,M6H 1M4,29,3.5,ON,['Let me start by saying that I love this plac...,other
9,6401 Morrison Blvd,"{'Alcohol': ""u'full_bar'"", 'BusinessAcceptsCre...",BvYU3jvGd0TJ7IyZdfiN2Q,"Sandwiches, Italian, American (Traditional), A...",Charlotte,"{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",0,35.156338,-80.831878,Manzetti's Tavern,28211,16,3.5,NC,"[""I had low country crab cakes for lunch becau...","Italian,American (Traditional),American (New),..."


In [34]:
cleanedData = newData.drop(['categories','hours','address','latitude','longitude','name','postal_code','state'], axis=1)
cleanedData

,attributes,business_id,city,is_open,review_count,stars,text,newCategory
0,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,Mississauga,1,128,2.5,"[""Went for Dim Sum when it was super packed. T...","Chinese,Seafood"
1,"{'GoodForKids': 'True', 'NoiseLevel': ""u'avera...",gnKjwL_1w79qoiV3IC_xQQ,Charlotte,1,170,4.0,"[""Lunch Review:\n\nWe pulled up at 11:45 am to...",Japanese
2,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",1Dfx3zM-rW4n-31KeC8sJg,Phoenix,1,18,3.0,['This location is consistently lightning fast...,Mexican
3,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...",fweCYi8FmbJXHCqLnwuk8w,Mentor-on-the-Lake,1,16,4.0,['This place is b.s. They sent me a $10 gift c...,Italian
4,"{'OutdoorSeating': 'False', 'BusinessAcceptsCr...",PZ-LZzSlhSe9utkQYU8pFg,Las Vegas,0,40,4.0,['I can\'t enough about this place as where to...,Italian
5,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",1RHY4K3BD22FK7Cfftn8Mg,Pittsburgh,1,35,4.0,"['My ""breakfast sandwich"" was a pathetic excus...",other
6,"{'BikeParking': 'True', 'BusinessParking': ""{'...",tstimHoMcYbkSC4eBA1wEg,Las Vegas,1,184,4.5,"[""Legit Mexican food from a small homey shop.\...",Mexican
7,"{'WiFi': ""u'no'"", 'BikeParking': 'True', 'Rest...",NDuUMJfrWk52RA-H-OtrpA,Toronto,1,57,3.0,"[""Not too shabby! This place is pretty good fo...",other
8,"{'BusinessParking': ""{'garage': False, 'street...",SP_YXIEwkFPPl_9anCYmpQ,Toronto,0,29,3.5,['Let me start by saying that I love this plac...,other
9,"{'Alcohol': ""u'full_bar'"", 'BusinessAcceptsCre...",BvYU3jvGd0TJ7IyZdfiN2Q,Charlotte,0,16,3.5,"[""I had low country crab cakes for lunch becau...","Italian,American (Traditional),American (New),..."


In [35]:
# check for missing values
for col in cleanedData.columns:
    print(col , " : ", cleanedData[col].isnull().sum())

attributes  :  901
business_id  :  0
city  :  0
is_open  :  0
review_count  :  0
stars  :  0
text  :  0
newCategory  :  0


In [37]:
(cleanedData['newCategory'] == 'other').sum()

21635

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

# silencing the warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# from nltk.corpus import stopwords
# stopWords = set(stopwords.words('english'))

nltk.download('stopwords')
stopWords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/dhivya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
cleanedData

,attributes,business_id,city,is_open,review_count,stars,text,newCategory
0,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,Mississauga,1,128,2.5,"[""Went for Dim Sum when it was super packed. T...","Chinese,Seafood"
1,"{'GoodForKids': 'True', 'NoiseLevel': ""u'avera...",gnKjwL_1w79qoiV3IC_xQQ,Charlotte,1,170,4.0,"[""Lunch Review:\n\nWe pulled up at 11:45 am to...",Japanese
2,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",1Dfx3zM-rW4n-31KeC8sJg,Phoenix,1,18,3.0,['This location is consistently lightning fast...,Mexican
3,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...",fweCYi8FmbJXHCqLnwuk8w,Mentor-on-the-Lake,1,16,4.0,['This place is b.s. They sent me a $10 gift c...,Italian
4,"{'OutdoorSeating': 'False', 'BusinessAcceptsCr...",PZ-LZzSlhSe9utkQYU8pFg,Las Vegas,0,40,4.0,['I can\'t enough about this place as where to...,Italian
5,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",1RHY4K3BD22FK7Cfftn8Mg,Pittsburgh,1,35,4.0,"['My ""breakfast sandwich"" was a pathetic excus...",other
6,"{'BikeParking': 'True', 'BusinessParking': ""{'...",tstimHoMcYbkSC4eBA1wEg,Las Vegas,1,184,4.5,"[""Legit Mexican food from a small homey shop.\...",Mexican
7,"{'WiFi': ""u'no'"", 'BikeParking': 'True', 'Rest...",NDuUMJfrWk52RA-H-OtrpA,Toronto,1,57,3.0,"[""Not too shabby! This place is pretty good fo...",other
8,"{'BusinessParking': ""{'garage': False, 'street...",SP_YXIEwkFPPl_9anCYmpQ,Toronto,0,29,3.5,['Let me start by saying that I love this plac...,other
9,"{'Alcohol': ""u'full_bar'"", 'BusinessAcceptsCre...",BvYU3jvGd0TJ7IyZdfiN2Q,Charlotte,0,16,3.5,"[""I had low country crab cakes for lunch becau...","Italian,American (Traditional),American (New),..."


In [40]:
tip = pd.read_json("tip.json", lines=True)
tip

,business_id,compliment_count,date,text,user_id
0,VaKXUpmWTTWDKbpJ3aQdMw,0,2014-03-27 03:51:24,"Great for watching games, ufc, and whatever el...",UPw5DWs_b-e2JRBS-t37Ag
1,OPiPeoJiv92rENwbq76orA,0,2013-05-25 06:00:56,Happy Hour 2-4 daily with 1/2 price drinks and...,Ocha4kZBHb4JK0lOWvE0sg
2,5KheTjYPu1HcQzQFtm4_vw,0,2011-12-26 01:46:17,Good chips and salsa. Loud at times. Good serv...,jRyO2V1pA4CdVVqCIOPc1Q
3,TkoyGi8J7YFjA6SbaRzrxg,0,2014-03-23 21:32:49,The setting and decoration here is amazing. Co...,FuTJWFYm4UKqewaosss1KA
4,AkL6Ous6A1atZejfZXn1Bg,0,2012-10-06 00:19:27,Molly is definately taking a picture with Sant...,LUlKtaM3nXd-E4N4uOk_fQ
5,kj1p6NwUGROOoqNySf0I1Q,0,2011-12-31 18:02:42,It's true! The drunken noodles are outrageous!,iAd8XaHxv05iXyEiZ62Ibw
6,0hudPyuCBlKg79OwKBw-eQ,0,2012-10-11 23:06:21,"Only worth a visit in the summer time, to take...",w8p4JW6-ERsbg8aB5wjEAg
7,r49iBfbnfoK7yt4rdsL_7g,0,2016-01-05 06:39:33,Hands down best AYCE in town! Love it!,CaFbX7topxdLpBJ_SNDfGQ
8,YCRCFBr6myPS49on2aS2pQ,0,2017-03-13 03:29:40,Sat in the drive thru this afternoon for 30 mi...,VLy6l8L72adD7iTBNqiPXw
9,RTBIiq-fJUXRsrzbL4MCqw,0,2013-02-08 20:55:15,Okay so I had a groupon for a nearby service c...,zrvk0-B91ahJRSB-UcAmIA


In [41]:
tip['tipText'] = tip['text']
tip = tip.drop('text', axis=1)
tip

,business_id,compliment_count,date,user_id,tipText
0,VaKXUpmWTTWDKbpJ3aQdMw,0,2014-03-27 03:51:24,UPw5DWs_b-e2JRBS-t37Ag,"Great for watching games, ufc, and whatever el..."
1,OPiPeoJiv92rENwbq76orA,0,2013-05-25 06:00:56,Ocha4kZBHb4JK0lOWvE0sg,Happy Hour 2-4 daily with 1/2 price drinks and...
2,5KheTjYPu1HcQzQFtm4_vw,0,2011-12-26 01:46:17,jRyO2V1pA4CdVVqCIOPc1Q,Good chips and salsa. Loud at times. Good serv...
3,TkoyGi8J7YFjA6SbaRzrxg,0,2014-03-23 21:32:49,FuTJWFYm4UKqewaosss1KA,The setting and decoration here is amazing. Co...
4,AkL6Ous6A1atZejfZXn1Bg,0,2012-10-06 00:19:27,LUlKtaM3nXd-E4N4uOk_fQ,Molly is definately taking a picture with Sant...
5,kj1p6NwUGROOoqNySf0I1Q,0,2011-12-31 18:02:42,iAd8XaHxv05iXyEiZ62Ibw,It's true! The drunken noodles are outrageous!
6,0hudPyuCBlKg79OwKBw-eQ,0,2012-10-11 23:06:21,w8p4JW6-ERsbg8aB5wjEAg,"Only worth a visit in the summer time, to take..."
7,r49iBfbnfoK7yt4rdsL_7g,0,2016-01-05 06:39:33,CaFbX7topxdLpBJ_SNDfGQ,Hands down best AYCE in town! Love it!
8,YCRCFBr6myPS49on2aS2pQ,0,2017-03-13 03:29:40,VLy6l8L72adD7iTBNqiPXw,Sat in the drive thru this afternoon for 30 mi...
9,RTBIiq-fJUXRsrzbL4MCqw,0,2013-02-08 20:55:15,zrvk0-B91ahJRSB-UcAmIA,Okay so I had a groupon for a nearby service c...


In [43]:
newTipData = tip.groupby('business_id',as_index=False)[['tipText']].aggregate(lambda x: list(x))
newTipData

,business_id,tipText
0,--1UhMGODdWsrMastO9DZw,"[Delicious! One of the best burritos, salsa an..."
1,--6MefnULPED_I942VcFNA,"[BBQ pork is sold out early on Saturday, $5 lu..."
2,--7zmmkVg-IMGaXbuVd0SQ,"[Good place to stop when traffic don't go., Fl..."
3,--9QQLMTbFzLJ_oT-ON3Xw,[Tip? Don't get your hair cut here. If they're...
4,--9e1ONYQuAa-CB_Rrw7Tw,"[Wine pairings for my prix fixe meal. Mmmm, *..."
5,--DaPTJW3-tB1vP-PfdTEg,[Awesome. Huge portions definitely worth a vi...
6,--DdmeR16TRb3LsjG0ejrQ,[Burger extravaganza last night was awesome. P...
7,--EX4rRznJrltyn-34Jz1w,[Newly Renovated! It's connected to White Barn...
8,--FBCX-N37CMYDfs790Bnw,"[Love my girls here, This place is bad @$$!! N..."
9,--GM_ORV2cYS-h38DSaCLw,[Their wings were super big and they were very...


In [46]:
businessReviewTipData = pd.merge(cleanedData, newTipData, how='inner',on='business_id')
businessReviewTipData

,attributes,business_id,city,is_open,review_count,stars,text,newCategory,tipText
0,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,Mississauga,1,128,2.5,"[""Went for Dim Sum when it was super packed. T...","Chinese,Seafood","[Spring & egg rolls are great, get them with e..."
1,"{'GoodForKids': 'True', 'NoiseLevel': ""u'avera...",gnKjwL_1w79qoiV3IC_xQQ,Charlotte,1,170,4.0,"[""Lunch Review:\n\nWe pulled up at 11:45 am to...",Japanese,"[Tonkatsu Ramen!!!, Ramen, Gyoza, miso soup an..."
2,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",1Dfx3zM-rW4n-31KeC8sJg,Phoenix,1,18,3.0,['This location is consistently lightning fast...,Mexican,[Had to call police to get a refund for my foo...
3,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...",fweCYi8FmbJXHCqLnwuk8w,Mentor-on-the-Lake,1,16,4.0,['This place is b.s. They sent me a $10 gift c...,Italian,[We order here all the time and is always perf...
4,"{'OutdoorSeating': 'False', 'BusinessAcceptsCr...",PZ-LZzSlhSe9utkQYU8pFg,Las Vegas,0,40,4.0,['I can\'t enough about this place as where to...,Italian,[They've just closed their doors! Tried stoppi...
5,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",1RHY4K3BD22FK7Cfftn8Mg,Pittsburgh,1,35,4.0,"['My ""breakfast sandwich"" was a pathetic excus...",other,"[Incredible incredible vegetarian omelette, wo..."
6,"{'BikeParking': 'True', 'BusinessParking': ""{'...",tstimHoMcYbkSC4eBA1wEg,Las Vegas,1,184,4.5,"[""Legit Mexican food from a small homey shop.\...",Mexican,[Best most authentic Mexican food in Las Vegas...
7,"{'WiFi': ""u'no'"", 'BikeParking': 'True', 'Rest...",NDuUMJfrWk52RA-H-OtrpA,Toronto,1,57,3.0,"[""Not too shabby! This place is pretty good fo...",other,[Loved the smoothie & salad. People serving co...
8,"{'BusinessParking': ""{'garage': False, 'street...",SP_YXIEwkFPPl_9anCYmpQ,Toronto,0,29,3.5,['Let me start by saying that I love this plac...,other,"[Great space, menu divided between vegan and m..."
9,"{'Alcohol': ""u'full_bar'"", 'BusinessAcceptsCre...",BvYU3jvGd0TJ7IyZdfiN2Q,Charlotte,0,16,3.5,"[""I had low country crab cakes for lunch becau...","Italian,American (Traditional),American (New),...","[Closed, Manzetti's is now Paco's Tacos and Te..."


In [59]:
import re
def findClassWithMaxOccurrences(data):
    for i in range(len(data)):
        classVals = data.loc[i,'newCategory'].split(',')
        reviewText = str(data.loc[i,'text'])
        tipText = str(data.loc[i,'tipText'])
        maxVal = -1
        maxString = classVals[0]
        for category in classVals:
            reviewTextCount = sum(1 for _ in re.finditer(r'\b%s\b' % re.escape(category), reviewText))
            tipTextCount = sum(1 for _ in re.finditer(r'\b%s\b' % re.escape(category), tipText))
            finalCount = reviewTextCount + tipTextCount
            if finalCount != 0 and maxVal < finalCount:
                maxVal = finalCount
                maxString = category
        data.loc[i,'class'] = maxString
    
    return data
cleanedFullData = findClassWithMaxOccurrences(businessReviewTipData)
cleanedFullData

,attributes,business_id,city,is_open,review_count,stars,text,newCategory,tipText,class
0,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,Mississauga,1,128,2.5,"[""Went for Dim Sum when it was super packed. T...","Chinese,Seafood","[Spring & egg rolls are great, get them with e...",Chinese
1,"{'GoodForKids': 'True', 'NoiseLevel': ""u'avera...",gnKjwL_1w79qoiV3IC_xQQ,Charlotte,1,170,4.0,"[""Lunch Review:\n\nWe pulled up at 11:45 am to...",Japanese,"[Tonkatsu Ramen!!!, Ramen, Gyoza, miso soup an...",Japanese
2,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",1Dfx3zM-rW4n-31KeC8sJg,Phoenix,1,18,3.0,['This location is consistently lightning fast...,Mexican,[Had to call police to get a refund for my foo...,Mexican
3,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...",fweCYi8FmbJXHCqLnwuk8w,Mentor-on-the-Lake,1,16,4.0,['This place is b.s. They sent me a $10 gift c...,Italian,[We order here all the time and is always perf...,Italian
4,"{'OutdoorSeating': 'False', 'BusinessAcceptsCr...",PZ-LZzSlhSe9utkQYU8pFg,Las Vegas,0,40,4.0,['I can\'t enough about this place as where to...,Italian,[They've just closed their doors! Tried stoppi...,Italian
5,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",1RHY4K3BD22FK7Cfftn8Mg,Pittsburgh,1,35,4.0,"['My ""breakfast sandwich"" was a pathetic excus...",other,"[Incredible incredible vegetarian omelette, wo...",other
6,"{'BikeParking': 'True', 'BusinessParking': ""{'...",tstimHoMcYbkSC4eBA1wEg,Las Vegas,1,184,4.5,"[""Legit Mexican food from a small homey shop.\...",Mexican,[Best most authentic Mexican food in Las Vegas...,Mexican
7,"{'WiFi': ""u'no'"", 'BikeParking': 'True', 'Rest...",NDuUMJfrWk52RA-H-OtrpA,Toronto,1,57,3.0,"[""Not too shabby! This place is pretty good fo...",other,[Loved the smoothie & salad. People serving co...,other
8,"{'BusinessParking': ""{'garage': False, 'street...",SP_YXIEwkFPPl_9anCYmpQ,Toronto,0,29,3.5,['Let me start by saying that I love this plac...,other,"[Great space, menu divided between vegan and m...",other
9,"{'Alcohol': ""u'full_bar'"", 'BusinessAcceptsCre...",BvYU3jvGd0TJ7IyZdfiN2Q,Charlotte,0,16,3.5,"[""I had low country crab cakes for lunch becau...","Italian,American (Traditional),American (New),...","[Closed, Manzetti's is now Paco's Tacos and Te...",Italian


In [60]:
cleanedFullData.groupby('class').size()

class
American (New)             2938
American (Traditional)     4812
Asian Fusion                489
Chinese                    3189
French                      700
Indian                      969
Italian                    3500
Japanese                   1769
Korean                      632
Mediterranean              1078
Mexican                    3702
Middle Eastern              550
Seafood                    1151
Thai                       1008
other                     18305
dtype: int64

In [78]:
cleanedFinalData = cleanedFullData[cleanedFullData['class'] != 'other']
cleanedFinalData = cleanedFinalData[cleanedFinalData['class'] != 'Middle Eastern']
cleanedFinalData = cleanedFinalData[cleanedFinalData['class'] != 'Asian Fusion']
cleanedFinalData = cleanedFinalData[cleanedFinalData['class'] != 'Korean']
cleanedFinalData = cleanedFinalData[cleanedFinalData['class'] != 'French']

print(len(cleanedFinalData))
print(cleanedFinalData.groupby('class').size())

24116
class
American (New)            2938
American (Traditional)    4812
Chinese                   3189
Indian                     969
Italian                   3500
Japanese                  1769
Mediterranean             1078
Mexican                   3702
Seafood                   1151
Thai                      1008
dtype: int64


In [79]:
cleanedFinalData.to_csv("Business_Review_Tip_Data.csv")